In [1]:
from utility_bfvariater import *
import datetime

In [2]:
tb = BFSolver_Toolbox()
er = BFVariater()

# 1. Try One Case

In [3]:
for i in range(4):

    ndata,ndataVar = er.ndata[i],er.ndataVar[i]
    nmcbg,nmcbgVar = er.nmcbg[i].dot(np.array([1,1,1])), er.nmcbgVar[i].dot(np.array([1,1,1]))
    nfake,nfakeVar = er.nfake[i],er.nfakeVar[i]
    a,aVar = er.a[i],er.aVar[i]
    aVar[a<0.001] = np.zeros_like(a)[a<0.001]

    slv0 = BFSolver3D(a)
    br0 = slv0.solveQuadEqn(slv0.setMeasuredX(nData=ndata, nMcbg=nmcbg+nfake))
    print(br0)

[0.10912736 0.1077028  0.10706577]
[0.11225778 0.11009797 0.10503398]
[0.10531882 0.10770093 0.10801755]
[0.11050912 0.10937777 0.10776729]


# 2. Statistical and Systematical Error

In [4]:
total = np.zeros([4,1,3])
# PDG
err = [ er.errConstent(source) for source in ["BtmUp","BteUp"] ]
err = np.array(err).transpose(1,0,2)
total = np.concatenate( (total,err),axis=1 )

# sysCrossSection
err = [ er.errSystem_crossSection(source) for source in ["mctt","mctw","mcw",'mcz','mcvv',"fakemu","fakee","faketau","lumin"] ]
err = np.array(err).transpose(1,0,2)
total = np.concatenate( (total,err),axis=1 )

# sysObjectEff
err = [ er.errSystem_objectEff(source) for source in ["e","mu","tauID",'jetMisTauID'] ]
#err = [ er.errSystem_objectEff(source) for source in ["e","mu",'tauIDNotFactorized'] ]
err = np.array(err).transpose(1,0,2)
total = np.concatenate( (total,err),axis=1 )

# sysEnergyScale
err = [ er.errSystem_energyScale(source) for source in ["E","Mu","Tau"] ]
err = np.array(err).transpose(1,0,2)
total = np.concatenate( (total,err),axis=1 )

# sysJet
err = [ er.errSystem_upDownVariation(source) for source in ["JES","JER","BTag","Mistag"] ]
err = np.array(err).transpose(1,0,2)
total = np.concatenate( (total,err),axis=1 )

er.io_printErrorForExcelFormat(total)

0.000,0.000,0.000, 0.000,0.000,0.000, 0.000,0.000,0.000, 0.000,0.000,0.000
0.005,0.050,0.076, 0.005,0.049,0.075, 0.008,0.050,0.076, 0.007,0.049,0.076
0.029,0.010,0.059, 0.028,0.010,0.058, 0.025,0.007,0.060, 0.024,0.008,0.060
0.010,0.003,0.105, 0.003,0.010,0.057, 0.011,0.008,0.114, 0.009,0.007,0.004
0.010,0.003,0.109, 0.003,0.011,0.059, 0.012,0.008,0.119, 0.009,0.007,0.004
0.156,0.156,0.149, 0.051,0.051,0.040, 0.150,0.152,0.168, 0.048,0.048,0.072
0.147,0.707,0.070, 0.049,0.172,0.036, 0.438,0.135,0.024, 0.055,0.042,0.022
0.000,0.020,0.012, 0.001,0.009,0.002, 0.016,0.001,0.008, 0.006,0.001,0.001
0.641,0.618,1.286, 0.423,0.403,0.708, 0.000,0.000,0.000, 0.000,0.000,0.000
0.000,0.000,0.000, 0.000,0.000,0.000, 0.377,0.379,0.785, 0.204,0.197,0.353
0.249,0.333,1.704, 0.149,0.196,1.020, 0.226,0.303,1.632, 0.102,0.137,0.750
0.151,0.280,0.037, 0.050,0.062,0.001, 0.148,0.144,0.070, 0.005,0.045,0.025
1.008,0.091,0.077, 1.027,0.093,0.075, 1.067,0.109,0.086, 1.016,0.102,0.080
0.140,1.184,0.122, 0.129,

In [5]:
# systematic
sigma_syst = total.transpose(1,0,2)
s = sigma_syst.shape
sigma_syst = sigma_syst.reshape(s[0],s[1]*s[2])

covar_syst = []
for i in sigma_syst:
    covar_syst.append(np.outer(i,i))
covar_syst = np.array(covar_syst)

# statistics
sig_stat, covar_stat_raw = er.errStat('data',returnCovar=True)
covar_stat = np.zeros([12,12])
for c in range(4):
    for i in range(3):
        for j in range(3):
            covar_stat[c*3+i,c*3+j] = covar_stat_raw[c,i,j]
            
            
covar = covar_syst
covar[0] = covar_stat

In [6]:
np.save('../data/combine/covar',covar)
np.save('../data/combine/sigma',sigma_syst) 

In [ ]:
# statistics
# err = [ er.errStat(source) for source in ["data","mcbg","mcsg"] ]
# err = np.array(err).transpose(1,0,2)
# er.io_printErrorForExcelFormat(err)
# total = np.concatenate( (total,err),axis=1 )

# # sysLHE
# err = [ er.errSystem_upDownVariation(source) for source in ["Renorm","Factor","PDF"] ]
# # err[0] = err[0]/(0.75/0.1)
# # err[1] = err[1]/(0.75/0.1)
# err = np.array(err).transpose(1,0,2)
# er.io_printErrorForExcelFormat(err)
# #total = np.concatenate( (total,err),axis=1 )

# # errSystem_ttTheory
# err = [ er.errSystem_upDownVariation(source) for source in [ "ISR","FSR","UE","MEPS"] ]
# # err[0] = err[0]/(0.1108*0.75/0.0011)
# # err[1] = err[1]/(0.1108*0.75/0.0011)
# err = np.array(err).transpose(1,0,2)
# er.io_printErrorForExcelFormat(err)
# #total = np.concatenate( (total,err),axis=1 )